In [ ]:
from efficientnet_pytorch import EfficientNet


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *

In [ ]:
model = EfficientNet.from_name('efficientnet-b5')

In [ ]:
model_name = 'efficientnet-b5'
image_size = EfficientNet.get_image_size(model_name)

In [ ]:
image_size

In [ ]:
model = EfficientNet.from_pretrained(model_name)


In [ ]:
model

In [ ]:
np.random.seed(42)


In [ ]:
src = (ImageList.from_folder(path='train-snakes-cropped').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
tfms=([rotate(degrees=(-90,90), p=0.8)],[])

In [ ]:
bs=8

In [ ]:
data = (src.transform(tfms, size=image_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=bs).normalize(imagenet_stats))

In [ ]:
data.show_batch(3,figsize=(9,9))

In [ ]:
list(children(model))[-2:]


In [ ]:
model

In [ ]:
ef_head = create_head(2048, data.c, concat_pool=False)
ef_head = nn.Sequential(*(list(ef_head.children())[3:]))

In [ ]:
model.add_module('_fc',nn.Linear(2048, data.c))
# model.add_module('_fc',ef_head)

In [ ]:
# loss_func =LabelSmoothingCrossEntropy()


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()
loss_func = FocalLoss(gamma=2.5)

In [ ]:
RMSprop = partial(torch.optim.RMSprop)


In [ ]:
from torch.optim import Optimizer

In [ ]:
learn = Learner(data, model, loss_func=loss_func, opt_func=RMSprop, metrics=[accuracy,FBeta(beta=1,average='macro')],wd=1e-5)


In [ ]:
learn

In [ ]:
learn.model

In [ ]:
learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:19]],
             [learn.model._blocks[19:],learn.model._conv_head], 
             [learn.model._bn1,learn.model._fc]])

In [ ]:
learn.summary()

In [ ]:
learn.load('b5-fl-round-2')

In [ ]:
learn.lr_find()


In [ ]:
learn.validate()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=1e-5

In [ ]:
learn.fit(2,lr)

In [ ]:
learn.save('b5-fl-round-3')

In [ ]:
learn.fit_one_cycle(2,slice(1e-7,1e-5))

In [ ]:
learn.save('b5-fl-round-4')

In [ ]:
learn.validate()